In [39]:
!pwd

/home/sun.jiam/worldstrat


In [3]:
!cd ../../scratch/sun.jiam

In [4]:
# !git clone https://github.com/worldstrat/worldstrat
%cd worldstrat
# !pip install -r requirements.txt

/home/sun.jiam/worldstrat


In [60]:
import os

from ipywidgets import interact_manual, Layout
import ipywidgets as widgets
from glob import glob
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

from src.datasets import load_dataset_JIF, SatelliteDataset, DictDataset, make_transforms_JIF
from src.datasources import S2_ALL_12BANDS
from src.plot import showtensor
from multiprocessing import Manager
from src.datasources import *
import warnings
import torch.nn.functional as F
from src.plot import showtensor
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
transforms = make_transforms_JIF(lr_bands_to_use='true_color', radiometry_depth=12)
multiprocessing_manager = Manager()

dataset_root = '../../../scratch/sun.jiam/dataset/'
hr_dataset_folder = 'hr_dataset/12bit/'
lr_dataset_folder = 'lr_dataset/'

In [53]:
from multiprocessing import Manager
from src.datasets import load_dataset_JIF
import pandas as pd

multiprocessing_manager = Manager()
list_of_aois = list(
    pd.read_csv("pretrained_model/stratified_train_val_test_split.csv", index_col=1).index
)
kws = dict(
    input_size=(400, 400),
    output_size=(1024, 1024),
    normalize_lr=True,
    root="../../../scratch/sun.jiam/dataset/",
    radiometry_depth=12,
    lr_bands_to_read="true_color",
)

dataloaders = load_dataset_JIF(**kws)

Using all LR bands.
Shuffling the dataset splits using 42
Train set size: 3143
Val set size: 392
Test set size: 392


In [55]:
train_dataset = dataloaders["train"].dataset
validation_dataset = dataloaders["val"].dataset
test_dataset = dataloaders["test"].dataset

In [21]:
# ! pip install einops

In [56]:
print(train_dataset[0]["lr"].shape)
print(train_dataset[0]["lrc"].shape)
print(train_dataset[0]["hr"].shape)

torch.Size([8, 12, 400, 400])
torch.Size([8, 1, 400, 400])
torch.Size([1, 3, 1024, 1024])


In [67]:
#Conv2d only takes 3d inputs or 4d where the first element is the batch
#We can keep the 12 channels, it is very slow to get only RGB. 
x = train_dataset[0]["lr"].reshape(256*8, 12, 25, 25)

In [65]:
#Mini test for 5d input, which didn't work. 
#We need to convert the 5d input to 3d. 
# 5d: 256, 8, 12, 25, 25
# 3d: 12, 25, 25
# The shape of the torch: (256 * 8, 12, 25, 25)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(12, 32, 5, 1)
    def forward(self, x):
        x = self.conv1(x)
        return x

In [68]:
#Conv2d only takes 3d inputs or 4d where the first element is the batch
c= ConvNet()
x = c(x[0])
x.shape

torch.Size([32, 21, 21])

In [57]:
# Still designing
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(12, 32, 5, 1)
        self.conv2 = nn.Conv2d(32, 128, 3, 3)
        self.conv3 = nn.Conv2d(128, 256, 11, 1)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.linear(6144, 1024)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        return x 

In [8]:
import torch
from torch import nn
from einops import rearrange


class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, last):
        super().__init__()
        if not last:
            last=dim
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, last),
        )
    def forward(self, x):
        return self.net(x)

    
class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64):
        super().__init__()
        inner_dim = dim_head *  heads
        self.heads = heads
        self.scale = dim_head ** -0.5
        self.norm = nn.LayerNorm(dim)

        self.attend = nn.Softmax(dim = -1)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Linear(inner_dim, dim, bias = False)

    def forward(self, x):
        x = self.norm(x)

        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

    
class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, out_dim):
        super().__init__()
        self.layers = nn.ModuleList([])
        self.depth = depth
        for d in range(depth):
            if d == depth-1:
                last=out_dim
            else: 
                last=None
            self.layers.append(nn.ModuleList([
                Attention(dim, heads = heads, dim_head = dim_head),
                FeedForward(dim, mlp_dim, last)
            ]))
    def forward(self, x):
        for d in range(self.depth):
            attn, ff = self.layers[d]
            x = attn(x) + x
            x = ff(x)
            if d!=self.depth-1:
                x+=x
        return x